In [2]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score,precision_score, recall_score, roc_auc_score
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.feature_selection import RFE

from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.mixture import GaussianMixture
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
import tensorflow.keras
import keras.metrics
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D
from keras.models import Sequential
from imblearn.under_sampling import RandomUnderSampler

from keras.callbacks import History 
from keras.utils import plot_model
from keras.optimizers import SGD
from sklearn.preprocessing import StandardScaler

In [3]:
path = "PTSD.xlsx"
df = pd.read_excel(path)
df = df[~df["PCL_Strict3"].isna()]


In [4]:
features =  [ "age", "highschool_diploma", "Hebrew", "dyslexia", "ADHD", "T1ETBE", "T1Acc1t",
                         "T1Acc1n", "T1bias", "T2Acc1t", "T2Acc1n", "T2bias", "state1", "state2", "trait1",
                         "trait2", "lot1", "lot2", "phq1", "phq2", "PCL1", "PCL2", "cd_risc1", "ptgi2",
                         "active_coping1", "planning1", "positive_reframing1", "acceptance1", "humor1",
                         "religion1", "emotional_support1", "instrumental_support1", "self_distraction1",
                         "denial1", "venting1", "substance_use1", "behavioral_disengagement1", "self_blame1",
                         "active_coping2", "planning2", "positive_reframing2", "acceptance2", "humor2",
                         "religion2", "emotional_support2", "instrumental_support2","self_distraction2",
                         "denial2", "venting2", "substance_use2", "behavioral_disengagement2", "self_blame2",
                         "trauma_history8_1", "military_exposure_unit", "HML_5HTT", "HL_MAOA", "HML_NPY",
                         "COMT_Hap1_recode", "COMT_Hap2_recode", "COMT_Hap1_LvsMH", "HML_FKBP5", "Ashken_scale",
                         "Sephar_scale", "Unknown", "bad_features"]

In [5]:
bad_features = ["T1ETBE", "T1bias", "state1", "state2", "trait1", "trait2", "phq1", "phq2", "PCL1", "PCL2",
                "denial1", "substance_use1", "self_blame1", "denial2", "substance_use2","self_blame2",
                         "trauma_history8_1", "military_exposure_unit", 'trauma6t2', 'trauma8t2',
                        't1bias_1_zero', 'state1_zero', 'trait1_zero', 'PHQ1_zero', 'PCL1_zero', 'depression_clinical2',
                        'ptsd1_clini', 'emotional_cop1n']
numerical_features = [ "T1ETBE", "T1Acc1t", "T1Acc1n", "T1bias", "T2Acc1t","T2Acc1n", "T2bias", "state1", "state2",
                        "trait1", "trait2", "lot1", "lot2", "phq1", "phq2", "cd_risc1", "PCL1", "PCL2"]
categorical_features = [ "age", "highschool_diploma", "Hebrew", "dyslexia", "ADHD",   "ptgi2",
                    "active_coping1", "planning1", "positive_reframing1", "acceptance1", "humor1",
                    "religion1", "emotional_support1", "instrumental_support1", "self_distraction1",
                    "denial1", "venting1", "substance_use1", "behavioral_disengagement1", "self_blame1",
                    "active_coping2", "planning2", "positive_reframing2", "acceptance2", "humor2",
                    "religion2", "emotional_support2", "instrumental_support2", "self_distraction2",
                    "denial2", "venting2", "substance_use2", "behavioral_disengagement2", "self_blame2",
                    "trauma_history8_1", "military_exposure_unit", "HML_5HTT", "HL_MAOA", "HML_NPY",
                    "COMT_Hap1_recode", "COMT_Hap2_recode", "COMT_Hap1_LvsMH", "HML_FKBP5", "Ashken_scale",
                    "Sephar_scale", "Unknown"]

In [6]:
df['bad_features'] = (df > df.mean())[bad_features].sum(axis=1)

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
df[numerical_features] = imp.fit_transform(df[numerical_features])

imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
df[categorical_features] = imp.fit_transform(df[categorical_features])

X = df[features]



Y = df["PCL3"]>50

In [7]:
sum(Y)

44

In [8]:
X["interaction_1"] = X["T1Acc1t"] * X["T2Acc1n"] * X["military_exposure_unit"]
X["interaction_2"] = X["T1Acc1n"] * X["T2Acc1t"] * X["military_exposure_unit"]
X["interaction_3"] = X["highschool_diploma"] * X["military_exposure_unit"] * X['PCL1']
X["interaction_4"] = X["T1ETBE"] * X["military_exposure_unit"] * X['HML_5HTT']


C:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.

In [9]:
ss = StandardScaler()
X = ss.fit_transform(X)

C:\Anaconda\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Anaconda\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.1, random_state=271828, stratify=Y)
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_train, y_train, test_size = 0.1, random_state=271828, stratify=y_train)
#X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X_train_2, y_train_2, test_size = 0.1, random_state=271828, stratify=y_train_2)

In [11]:
def create_data(X_train, y_train):
    X_train_3 = X_train[y_train==1]
    y_train_3 = y_train[y_train==1]
    X_train_4 = X_train[y_train==0][:10:]
    y_train_4 = y_train[y_train==0][:10:]
    X_train_5 = np.vstack((X_train_4, X_train_3))
    y_train_5 =  np.hstack((y_train_4, y_train_3))
    sm = SMOTE(random_state=27)
    X_train_6, y_train_6 = sm.fit_sample(X_train_5, y_train_5.ravel())
    X_train_6 = X_train_6[y_train_6==0]
    y_train_6 = y_train_6[y_train_6==0]
    return X_train_6, y_train_6

In [12]:
preds = []
trues= []

In [13]:
kfold = StratifiedKFold(n_splits=7, shuffle=True)
cvscores = []
y_train_2 = np.array(y_train_2)
X_train_2 = np.array(X_train_2)

for num_layers in [2,1, 0]:
    for first_layer in [75, 25]:
        for rfe_features in [ 6, 12, 20]:
            for class_weight in [{1:1, 0:5}, {1:1, 0:10}, {1:1, 0:2.5}]:
                num_layers = num_layers
                first_layer = first_layer
                each_layer = 10
                num_smote = 1
                loops = 0
                scores_f = []
                scores_r = []
                scores_p = []
                print("num_layers", num_layers, "\nfirst_layer", first_layer, 
                          "\nclass_weight", class_weight, "\nnum_smote", num_smote, "\nrfe", rfe_features)
                for train, test in kfold.split(X_train_2, y_train_2):

                    sm = SMOTE(k_neighbors= 10)
                    X_train_res = X_train_2[train]
                    y_train_res = y_train_2[train]
                    
                    rfe =  RFE(RandomForestClassifier(n_estimators=100), rfe_features).fit(X_train_res, y_train_res)
                    X_train_rfe = rfe.transform(X_train_2[train])
                    X_test_rfe = rfe.transform(X_train_2[test])
                    
                    X_train_res, y_train_res = sm.fit_sample(X_train_rfe, y_train_res.ravel())

                  # create model
                    n_cols = X_train_res.shape[1]
                    model = Sequential()
                    model.add(Dense(first_layer, activation='elu', input_dim = n_cols))
                    model.add(Dropout(0.5))

                    for i in range(num_layers):
                        model.add(Dense(each_layer, activation='elu'))
                        model.add(Dropout(0.5))

                    model.add(Dense(1, activation='sigmoid'))

                    model.compile(optimizer='adam', 
                                  loss='binary_crossentropy')


                    # Fit the model
                    callbacks = [EarlyStopping(monitor='val_loss', patience=0)]
                    model.fit(X_train_res, y_train_res, epochs = 400, validation_split = .1, verbose=0, callbacks=callbacks, class_weight = class_weight)
                    # evaluate the model
                    y_pred =  model.predict(X_test_rfe)
                    y_train_pred =  model.predict(X_train_rfe)
                    preds.extend(y_pred)
                    trues.extend(y_train_2[test])
                    y_pred = y_pred>0.5
                    s_f = f1_score(y_train_2[test], y_pred)
                    s_p = precision_score(y_train_2[test], y_pred)
                    s_r = recall_score(y_train_2[test], y_pred)
                    print("\tscores f1", (s_f))
                    print("\tscores p", (s_p))
                    print("\tscores r", (s_r))
                    s = f1_score(y_pred,y_train_2[test])
                    #print(s)
                    
                    scores_f.append(s_f)
                    scores_p.append(s_p)
                    scores_r.append(s_r)

                    
                    y_train_pred = (y_train_pred > 0.5)
                    train_s_f = f1_score(y_train_2[train], y_train_pred)
                    train_s_p = precision_score(y_train_2[train], y_train_pred)
                    train_s_r = recall_score(y_train_2[train], y_train_pred)
                    print("\tscores f1 train", (train_s_f))
                    print("\tscores p train", (train_s_p))
                    print("\tscores r train", (train_s_r))
                print("mean precision scores", np.mean(scores_p))

num_layers 2 
first_layer 75 
class_weight {1: 1, 0: 5} 
num_smote 1 
rfe 6
	scores f1 0.25
	scores p 0.2
	scores r 0.3333333333333333
	scores f1 train 0.3469387755102041
	scores p train 0.25
	scores r train 0.5666666666666667
	scores f1 0.4615384615384615
	scores p 0.375
	scores r 0.6
	scores f1 train 0.2857142857142857
	scores p train 0.208955223880597
	scores r train 0.45161290322580644
	scores f1 0.13333333333333333
	scores p 0.1
	scores r 0.2
	scores f1 train 0.35714285714285715
	scores p train 0.24691358024691357
	scores r train 0.6451612903225806
	scores f1 0.4000000000000001
	scores p 0.4
	scores r 0.4
	scores f1 train 0.1971830985915493
	scores p train 0.175
	scores r train 0.22580645161290322
	scores f1 0.13333333333333333
	scores p 0.1
	scores r 0.2
	scores f1 train 0.3333333333333333
	scores p train 0.2641509433962264
	scores r train 0.45161290322580644
	scores f1 0.2727272727272727
	scores p 0.17647058823529413
	scores r 0.6
	scores f1 train 0.3181818181818182
	scores p tr

C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


	scores f1 0.0
	scores p 0.0
	scores r 0.0
	scores f1 train 0.0
	scores p train 0.0
	scores r train 0.0
	scores f1 0.12500000000000003
	scores p 0.09090909090909091
	scores r 0.2
	scores f1 train 0.2285714285714286
	scores p train 0.20512820512820512
	scores r train 0.25806451612903225
	scores f1 0.4000000000000001
	scores p 0.4
	scores r 0.4
	scores f1 train 0.2456140350877193
	scores p train 0.2692307692307692
	scores r train 0.22580645161290322
	scores f1 0.37499999999999994
	scores p 0.2727272727272727
	scores r 0.6
	scores f1 train 0.28205128205128205
	scores p train 0.23404255319148937
	scores r train 0.3548387096774194
	scores f1 0.0
	scores p 0.0
	scores r 0.0
	scores f1 train 0.0
	scores p train 0.0
	scores r train 0.0
	scores f1 0.28571428571428575
	scores p 0.5
	scores r 0.2
	scores f1 train 0.16666666666666666
	scores p train 0.23529411764705882
	scores r train 0.12903225806451613
	scores f1 0.20000000000000004
	scores p 0.2
	scores r 0.2
	scores f1 train 0.3333333333333333

	scores f1 0.2727272727272727
	scores p 0.17647058823529413
	scores r 0.6
	scores f1 train 0.25196850393700787
	scores p train 0.16666666666666666
	scores r train 0.5161290322580645
	scores f1 0.14285714285714285
	scores p 0.1111111111111111
	scores r 0.2
	scores f1 train 0.24096385542168675
	scores p train 0.19230769230769232
	scores r train 0.3225806451612903
	scores f1 0.0
	scores p 0.0
	scores r 0.0
	scores f1 train 0.3333333333333333
	scores p train 0.3142857142857143
	scores r train 0.3548387096774194
	scores f1 0.08695652173913045
	scores p 0.05555555555555555
	scores r 0.2
	scores f1 train 0.1263157894736842
	scores p train 0.09375
	scores r train 0.1935483870967742
	scores f1 0.4
	scores p 0.26666666666666666
	scores r 0.8
	scores f1 train 0.1935483870967742
	scores p train 0.14516129032258066
	scores r train 0.2903225806451613
mean precision scores 0.12125350140056024
num_layers 2 
first_layer 75 
class_weight {1: 1, 0: 2.5} 
num_smote 1 
rfe 20
	scores f1 0.30769230769230765

C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


...

	scores f1 0.33333333333333337
	scores p 0.23076923076923078
	scores r 0.6
	scores f1 train 0.2393162393162393
	scores p train 0.16279069767441862
	scores r train 0.45161290322580644
	scores f1 0.3
	scores p 0.2
	scores r 0.6
	scores f1 train 0.3103448275862069
	scores p train 0.21176470588235294
	scores r train 0.5806451612903226
	scores f1 0.06451612903225806
	scores p 0.038461538461538464
	scores r 0.2
	scores f1 train 0.2763157894736842
	scores p train 0.17355371900826447
	scores r train 0.6774193548387096
mean precision scores 0.19469139882673717
num_layers 1 
first_layer 25 
class_weight {1: 1, 0: 5} 
num_smote 1 
rfe 12
	scores f1 0.06451612903225806
	scores p 0.04
	scores r 0.16666666666666666
	scores f1 train 0.15476190476190477
	scores p train 0.09420289855072464
	scores r train 0.43333333333333335
	scores f1 0.2608695652173913
	scores p 0.16666666666666666
	scores r 0.6
	scores f1 train 0.1509433962264151
	scores p train 0.10666666666666667
	scores r train 0.2580645161290322

	scores f1 0.3157894736842105
	scores p 0.21428571428571427
	scores r 0.6
	scores f1 train 0.2937062937062937
	scores p train 0.1875
	scores r train 0.6774193548387096
mean precision scores 0.12682331493391402
num_layers 0 
first_layer 75 
class_weight {1: 1, 0: 5} 
num_smote 1 
rfe 6
	scores f1 0.125
	scores p 0.1
	scores r 0.16666666666666666
	scores f1 train 0.3333333333333333
	scores p train 0.2222222222222222
	scores r train 0.6666666666666666
	scores f1 0.14285714285714285
	scores p 0.1111111111111111
	scores r 0.2
	scores f1 train 0.3902439024390244
	scores p train 0.3137254901960784
	scores r train 0.5161290322580645
	scores f1 0.23529411764705882
	scores p 0.16666666666666666
	scores r 0.4
	scores f1 train 0.3333333333333333
	scores p train 0.23943661971830985
	scores r train 0.5483870967741935
	scores f1 0.33333333333333337
	scores p 0.23076923076923078
	scores r 0.6
	scores f1 train 0.3137254901960784
	scores p train 0.22535211267605634
	scores r train 0.5161290322580645
	sc

	scores f1 0.3
	scores p 0.2
	scores r 0.6
	scores f1 train 0.2601626016260163
	scores p train 0.17391304347826086
	scores r train 0.5161290322580645
	scores f1 0.4
	scores p 0.3
	scores r 0.6
	scores f1 train 0.23728813559322032
	scores p train 0.16091954022988506
	scores r train 0.45161290322580644
	scores f1 0.4
	scores p 0.3
	scores r 0.6
	scores f1 train 0.24719101123595505
	scores p train 0.1896551724137931
	scores r train 0.3548387096774194
	scores f1 0.16666666666666666
	scores p 0.10526315789473684
	scores r 0.4
	scores f1 train 0.29357798165137616
	scores p train 0.20512820512820512
	scores r train 0.5161290322580645
	scores f1 0.1111111111111111
	scores p 0.07692307692307693
	scores r 0.2
	scores f1 train 0.26548672566371684
	scores p train 0.18292682926829268
	scores r train 0.4838709677419355
	scores f1 0.2
	scores p 0.13333333333333333
	scores r 0.4
	scores f1 train 0.3259259259259259
	scores p train 0.21153846153846154
	scores r train 0.7096774193548387
mean precision sc

	scores f1 0.11764705882352941
	scores p 0.08333333333333333
	scores r 0.2
	scores f1 train 0.2649006622516556
	scores p train 0.16666666666666666
	scores r train 0.6451612903225806
	scores f1 0.12500000000000003
	scores p 0.09090909090909091
	scores r 0.2
	scores f1 train 0.26229508196721313
	scores p train 0.17582417582417584
	scores r train 0.5161290322580645
	scores f1 0.08000000000000002
	scores p 0.05
	scores r 0.2
	scores f1 train 0.1839080459770115
	scores p train 0.11188811188811189
	scores r train 0.5161290322580645
	scores f1 0.20689655172413793
	scores p 0.125
	scores r 0.6
	scores f1 train 0.1971830985915493
	scores p train 0.11538461538461539
	scores r train 0.6774193548387096
mean precision scores 0.10281385281385282
num_layers 0 
first_layer 25 
class_weight {1: 1, 0: 10} 
num_smote 1 
rfe 12
	scores f1 0.20689655172413793
	scores p 0.13043478260869565
	scores r 0.5
	scores f1 train 0.20289855072463767
	scores p train 0.12962962962962962
	scores r train 0.46666666666666

KeyboardInterrupt: 

In [ ]:
preds = np.array(preds)
trues = np.array(trues)

In [ ]:
trues[np.where(preds > 0.7)[0]]

In [ ]:
num_layers = 2
first_layer = 75
each_layer = 10

# create model
n_cols = X_train_res.shape[1]
model = Sequential()
model.add(Dense(first_layer, activation='elu', input_dim = n_cols))
model.add(Dropout(0.5))
for i in range(num_layers):
    model.add(Dense(each_layer, activation='elu'))
    model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
                                  loss='binary_crossentropy')


                    # Fit the model
callbacks = [EarlyStopping(monitor='val_loss', patience=5)]
model.fit(X_train_2, y_train_2, epochs = 150, validation_split = .1, verbose=0, callbacks=callbacks, class_weight = {1:25, 0:75})
                    # evaluate the model
y_pred =  model.predict(X_test_2)
y_train_pred =  model.predict(X_train_2)
#preds.extend(y_pred)
    #trues.extend(y_train_2[test])
y_pred = y_pred>0.45
s_f = f1_score(y_test_2, y_pred)
s_p = precision_score(y_test_2, y_pred)
s_r = recall_score(y_test_2, y_pred)
print("\tscores f1", (s_f))
print("\tscores p", (s_p))
print("\tscores r", (s_r))
                    
y_train_pred = (y_train_pred) > 0.45
train_s_f = f1_score(y_train_2, y_train_pred)
train_s_p = precision_score(y_train_2, y_train_pred)
train_s_r = recall_score(y_train_2, y_train_pred)
print("\tscores f1 train", (train_s_f))
print("\tscores p train", (train_s_p))
print("\tscores r train", (train_s_r))
    